In [10]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('Amazon_Unlocked_Mobile.csv')

# Initialize an empty list to store the formatted strings
formatted_list = []

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Format the string as required
    formatted_string = f'Phone Name: {row["Product Name"]}\nReview: {row["Reviews"]}'
    # Append the formatted string to the list
    formatted_list.append(formatted_string)



In [11]:
import random
from qdrant_client import QdrantClient

# Initialize the client
client = QdrantClient(":memory:")

import random

def add_documents_in_batches(client, collection_name, formatted_list, batch_size=1000):
    """
    Adds documents to the collection in batches and prints a message after each batch.

    Parameters:
    client: The client object to interact with the database.
    collection_name (str): The name of the collection.
    formatted_list (list): The list of documents to add.
    batch_size (int): The number of documents to add in each batch. Default is 1000.
    """
    # Shuffle and sample the list
    sampled_list = random.sample(formatted_list, 2000)

    # Iterate over the sampled list in chunks of batch_size
    for i in range(0, len(sampled_list), batch_size):
        batch = sampled_list[i:i + batch_size]
        batch_ids = list(range(i + 1, i + 1 + len(batch)))

        # Add the batch to the collection
        client.add(
            collection_name=collection_name,
            documents=batch,
            ids=batch_ids
        )

        # Print a message indicating the batch has been added
        print(f"Batch {i // batch_size + 1} added with {len(batch)} documents.")

# Example usage
# Assuming `client` is your database client and `formatted_list` contains your documents
add_documents_in_batches(client, "phone_collection", formatted_list, batch_size=1000)

Batch 1 added with 1000 documents.
Batch 2 added with 1000 documents.


In [9]:
from dspy.retrieve.qdrant_rm import QdrantRM
import dspy


qdrant_retriever_model = QdrantRM("phone_collection", client)


ollama_model = dspy.OllamaLocal(model="llama3",model_type='text',
                                max_tokens=350,
                                temperature=0.1,
                                top_p=0.8, frequency_penalty=1.17, top_k=40)


dspy.settings.configure(lm= ollama_model, rm=qdrant_retriever_model)

In [7]:
class GenerateAnswer(dspy.Signature):
    """Generate a numbered list of phone names"""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(format=list)

In [8]:
class GenerateSearchQuery(dspy.Signature):
    """Write a simple search query that will help answer a complex question."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    query = dspy.OutputField()

In [5]:
from dsp.utils import deduplicate

class SimplifiedBaleen(dspy.Module):
    def __init__(self, passages_per_hop=3, max_hops=2):
        super().__init__()

        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops

    def forward(self, question):
        context = []

        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).query
            passages = self.retrieve(query).passages
            context = deduplicate(context + passages)

        pred = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=pred.answer)

In [12]:
# Ask any question you like to this simple RAG program.
my_question = "Phones with high quality camera and highly praised looks"

# Get the prediction. This contains `pred.context` and `pred.answer`.
uncompiled_baleen = SimplifiedBaleen()  # uncompiled (i.e., zero-shot) program
pred = uncompiled_baleen(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

Question: Phones with high quality camera and highly praised looks
Predicted Answer: Here is the numbered list of phone names that meet your criteria:

1. **BLU S480U** (Gold) - 5MP Main Camera
2. **BLU Studio Selfie** (Grey)

Note: These are the only two phones in the original list that have high-quality cameras and are highly praised for their looks, according to the reviews provided.
Retrieved Contexts (truncated): ['Phone Name: BLU S480U Unlocked Studio 7.0 II Smartphone with 5MP Main Camera (Gold)\nReview: Good product... Good transaction......', 'Phone Name: BLU S480U Unlocked Studio 7.0 II Smartphone with 5MP Main Camera (Gold)\nReview: This phone is all that and more dont be stupid not to get it and please dont listen t...', 'Phone Name: BLU Energy X Plus Smartphone - With 4000 mAh Super Battery- Global GSM Unlocked - Grey\nReview: Internet is slo. Pictures quickly is so bad...', 'Phone Name: BLU Studio C 5.0-Inch Smartphone with Android Lollipop OS - Unlocked (Grey)\nReview: E

In [13]:
ollama_model.inspect_history(n=3)




Write a simple search query that will help answer a complex question.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the query}. We ...

Query: ${query}

---

Context:
[1] «Phone Name: BLU S480U Unlocked Studio 7.0 II Smartphone with 5MP Main Camera (Gold)
Review: Good product... Good transaction...»
[2] «Phone Name: BLU S480U Unlocked Studio 7.0 II Smartphone with 5MP Main Camera (Gold)
Review: This phone is all that and more dont be stupid not to get it and please dont listen t»
[3] «Phone Name: BLU Energy X Plus Smartphone - With 4000 mAh Super Battery- Global GSM Unlocked - Grey
Review: Internet is slo. Pictures quickly is so bad»

Question: Phones with high quality camera and highly praised looks

Reasoning: Let's think step by step in order to Here is the search query that can help answer the complex question:

---

Context:
[1] «Phone Name: BLU S480U Unlocked Studio 7.0 

'\n\n\nWrite a simple search query that will help answer a complex question.\n\n---\n\nFollow the following format.\n\nContext: may contain relevant facts\n\nQuestion: ${question}\n\nReasoning: Let\'s think step by step in order to ${produce the query}. We ...\n\nQuery: ${query}\n\n---\n\nContext:\n[1] «Phone Name: BLU S480U Unlocked Studio 7.0 II Smartphone with 5MP Main Camera (Gold)\nReview: Good product... Good transaction...»\n[2] «Phone Name: BLU S480U Unlocked Studio 7.0 II Smartphone with 5MP Main Camera (Gold)\nReview: This phone is all that and more dont be stupid not to get it and please dont listen t»\n[3] «Phone Name: BLU Energy X Plus Smartphone - With 4000 mAh Super Battery- Global GSM Unlocked - Grey\nReview: Internet is slo. Pictures quickly is so bad»\n\nQuestion: Phones with high quality camera and highly praised looks\n\nReasoning: Let\'s think step by step in order to\x1b Here is the search query that can help answer the complex question:\n\n---\n\nContext:\n[1] «P